# Análise de sentimento usando word embeddings - IMDB

Anteriormente vimos uma primeira solução de análise de sentimento utilizando *bag of words*.
Agora iremos ilustrar o uso de *word embeddings* como vetor de atributos latentes de cada palavra.

Duas soluções são propostas neste exercícios:

1. Utilizando rede neural com camadas densas
2. Utilizando camadas convolucionais 1D

Diferentemente da solução apresentada com *bag of words*, nestas duas soluções, é necessário que o
número de palavras seja o mesmo para cada amostra. Para isso, limita-se o número de palavras e caso
o número de palavras for menor, completa-se com um código especial e palavras além do limite são
descartadas.

## Importação dos pacotes

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import os, sys
import json
import numpy as np
import pandas as pd
import numpy.random as nr

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import MultiStepLR, StepLR
from torch.utils.data import DataLoader, TensorDataset
from torch.autograd import Variable

from torchvision import datasets, transforms, models

import lib.pytorch_trainer as ptt

use_gpu = torch.cuda.is_available()
print('GPU available:', use_gpu)

GPU available: True


## Dataset IMDB

### Lendo do disco

O dataset é composto de 25 mil amostras de treinamento e 25 mil amostras de teste.
Cada amostra possui um texto de tamanho que varia entre 11 e 2494 palavras. 
Cada amostra tem um rótulo igual a 1 para denominar sentimento positivo e 0 para sentimento negativo.

In [2]:
word_index = json.load(open('/data/datasets/IMDB/imdb_word_index.json'))
data = np.load('/data/datasets/IMDB/imdb.npz')
x_test, x_train, y_train, y_test = data['x_test'], data['x_train'], data['y_train'], data['y_test']

n_words = len(word_index)
n_train = x_train.shape[0]
n_test  = x_test.shape[0]

word_list = [None for i in range(n_words+1)]
for k, v in word_index.items():
    word_list[v] = k

n_words, n_train, n_test

(88584, 25000, 25000)

In [3]:
def print_stats(x_train, x_test, word_list=None):
    print('Train word index limits:', min([min(s) for s in x_train]), max([max(s) for s in x_train]))
    print('Test word index limits:', min([min(s) for s in x_test]), max([max(s) for s in x_test]))
    print('\nTrain sequence length limits:', min([len(x) for x in x_train]), max([len(x) for x in x_train]))
    print('Test sequence length limits:', min([len(x) for x in x_test]), max([len(x) for x in x_test]))
    if word_list:
        print('\nMost frequent words:', word_list[1:11])
    
print_stats(x_train, x_test, word_list)

Train word index limits: 1 88584
Test word index limits: 1 88581

Train sequence length limits: 10 2493
Test sequence length limits: 6 2314

Most frequent words: ['the', 'and', 'a', 'of', 'to', 'is', 'br', 'in', 'it', 'i']


### Limitando o vocabulário

Retiramos das sequências as palavras com índice maior que o valor especificado em `voc_size`.

In [4]:
voc_size = 5000

xtra = [[w for w in x if (w < voc_size)] for x in x_train]
xval = [[w for w in x if (w < voc_size)] for x in x_test]
print_stats(xtra, xval)

Train word index limits: 1 4999
Test word index limits: 1 4999

Train sequence length limits: 9 1973
Test sequence length limits: 6 2113


### Obtendo sequências de mesmo comprimento

Fazemos com que todas as sequências tenham o mesmo comprimento, especificado em `seq_len`. As sequências mais longas que `seq_len` são truncadas e as menores, completadas com zeros.

In [5]:
def pad_sequences(sequences, seq_len, post_pad=True, fill_value=0):
    new_seq = []
    for seq in sequences:
        n = len(seq)
        if n > seq_len:
            if post_pad:
                new_seq.append(seq[-seq_len:])
            else:
                new_seq.append(seq[:seq_len])
        else:
            zseq = [fill_value for i in range(seq_len)]
            if post_pad:
                zseq[-n:] = seq
            else:
                zseq[:n] = seq
            new_seq.append(zseq)
    return new_seq
    

In [6]:
seq_len = 500
xtra = pad_sequences(xtra, seq_len, post_pad=True)
xval = pad_sequences(xval, seq_len, post_pad=True)
print_stats(xtra, xval)

Train word index limits: 0 4999
Test word index limits: 0 4999

Train sequence length limits: 500 500
Test sequence length limits: 500 500


### Convertendo para tensores

In [7]:
Xtrain = torch.from_numpy(np.array(xtra, np.int))
Xvalid = torch.from_numpy(np.array(xval, np.int))
ytrain = torch.from_numpy(np.array(y_train, np.int))
yvalid = torch.from_numpy(np.array(y_test, np.int))

Xtrain.size(), Xtrain.max(), Xvalid.size(), Xvalid.max()

(torch.Size([25000, 500]), 4999, torch.Size([25000, 500]), 4999)

## Rede Recorrente RNN


In [8]:
class ModelRNN(nn.Module):
    
    def __init__(self, hidden_size, voc_size=voc_size, embed_dim = None):
        super().__init__()
        self.emb = nn.Embedding(voc_size, embed_dim)
        nn.init.xavier_uniform(self.emb.weight)
        self.rnn = nn.RNN(input_size=embed_dim,hidden_size=hidden_size, dropout=0.05,batch_first=True)
        self.fc1 = nn.Linear(hidden_size,2)
        
    def forward(self, x):
        x = self.emb(x)
        _,x = self.rnn(x)
        x = self.fc1(x)
        x = torch.squeeze(x,0)
        return x
    
model_rnn = ModelRNN(100,5000,50)
if use_gpu:
    model_rnn = model_rnn.cuda()
model_rnn

ModelRNN (
  (emb): Embedding(5000, 50)
  (rnn): RNN(50, 100, batch_first=True, dropout=0.05)
  (fc1): Linear (100 -> 2)
)

## Testando predict com uma amostra

In [9]:
print(Xtrain[0:2].size())
xin = Variable(Xtrain[0:2])
if use_gpu:
    xin = xin.cuda()
y = model_rnn(xin)
y

torch.Size([2, 500])


Variable containing:
-0.1134 -0.0153
-0.1054 -0.0098
[torch.cuda.FloatTensor of size 2x2 (GPU 0)]

### Treinamento

In [10]:
embedding_dim = 50
batch_size = 100

# Callbacks
# ---------
state_fn = '../../models/sentimento_rnn'
accuracy_cb = ptt.AccuracyMetric()
chkpt_cb = ptt.ModelCheckpoint(state_fn, reset=True, verbose=1)
print_cb = ptt.PrintCallback()
plot_cb = ptt.PlotCallback()

# Model, optimizer and learning rate scheduler
# --------------------------------------------
optimizer = torch.optim.Adam(model_rnn.parameters(), lr=1e-4, weight_decay=0.0005)
scheduler = StepLR(optimizer, step_size=5, gamma=0.75)

# Network trainer
# ---------------
training_parameters = {
    'model':         model_rnn, 
    'criterion':     nn.CrossEntropyLoss(),
    'optimizer':     optimizer, 
    'lr_scheduler':  scheduler, 
    'callbacks':     [accuracy_cb, print_cb, chkpt_cb],
}
trainer = ptt.DeepNetTrainer(**training_parameters)


In [11]:
if False:
    trainer.fit(10, Xtrain, ytrain, valid_data=(Xvalid, yvalid), batch_size=batch_size)
else:
    trainer.load_state('/data/models/sentimento_rnn')

Treinando na GPU:

        Start training for 10 epochs
          1:  16.2s   T: 0.69239 0.52684   V: 0.69060 0.58852 best
          2:  16.2s   T: 0.64847 0.64060   V: 0.46676 0.78716 best
          3:  16.1s   T: 0.38345 0.83408   V: 0.36423 0.84764 best
          4:  16.2s   T: 0.30459 0.87692   V: 0.32521 0.86508 best
          5:  16.1s   T: 0.26332 0.89768   V: 0.31838 0.86872 best
          6:  16.1s   T: 0.25598 0.90048   V: 0.31837 0.86880 best
          7:  16.2s   T: 0.23778 0.90864   V: 0.36216 0.85484 
          8:  16.1s   T: 0.22960 0.91344   V: 0.33566 0.86748 
          9:  16.1s   T: 0.21615 0.92124   V: 0.33796 0.86820 
         10:  16.2s   T: 0.22252 0.91800   V: 0.34421 0.85672 
        Stop training at epoch: 10/10
        Best model was saved at epoch 6 with loss 0.31837: ../../models/sentimento_rnn

### Avaliação

In [12]:
if 'ModelCheckpoint' in [cb.__class__.__name__ for cb in trainer.callbacks]:
    trainer.load_state(state_fn)

rmetrics = trainer.evaluate(Xvalid, yvalid, metrics=[accuracy_cb])
print('Model validation set accuracy after training: {:.5f}'.format(rmetrics['acc']))

evaluate: 2499/2499 ok
Model validation set accuracy after training: 0.55732


## Rede Recorrente LSTM

In [13]:
class ModelLSTM(nn.Module):
    
    def __init__(self, hidden_size, voc_size=voc_size, embed_dim = None):
        super().__init__()
        self.emb = nn.Embedding(voc_size, embed_dim)
        nn.init.xavier_uniform(self.emb.weight)
        self.lstm = nn.LSTM(input_size=embed_dim,hidden_size=hidden_size, dropout=0.05,batch_first=True)
        self.fc1 = nn.Linear(hidden_size,2)
        
    def forward(self, x):
        x = self.emb(x)
        _,(x,_) = self.lstm(x)
        x = self.fc1(x)
        x = torch.squeeze(x,0)
        return x
    
model_lstm = ModelLSTM(100,5000,50)
if use_gpu:
    model_lstm = model_lstm.cuda()
model_lstm

ModelLSTM (
  (emb): Embedding(5000, 50)
  (lstm): LSTM(50, 100, batch_first=True, dropout=0.05)
  (fc1): Linear (100 -> 2)
)

## Testando predict com uma amostra

In [14]:
print(Xtrain[0:2].size())
xin = Variable(Xtrain[0:2])
if use_gpu:
    xin = xin.cuda()
y = model_lstm(xin)
y

torch.Size([2, 500])


Variable containing:
 0.1110 -0.0229
 0.1122 -0.0199
[torch.cuda.FloatTensor of size 2x2 (GPU 0)]

### Treinamento

In [15]:
embedding_dim = 50
batch_size = 100

# Callbacks
# ---------
state_fn = '../../models/sentimento_lstm'
accuracy_cb = ptt.AccuracyMetric()
chkpt_cb = ptt.ModelCheckpoint(state_fn, reset=True, verbose=1)
print_cb = ptt.PrintCallback()
plot_cb = ptt.PlotCallback()

# Model, optimizer and learning rate scheduler
# --------------------------------------------
optimizer = torch.optim.Adam(model_lstm.parameters(), lr=1e-4, weight_decay=0.0005)
scheduler = StepLR(optimizer, step_size=5, gamma=0.75)

# Network trainer
# ---------------
training_parameters = {
    'model':         model_lstm, 
    'criterion':     nn.CrossEntropyLoss(),
    'optimizer':     optimizer, 
    'lr_scheduler':  scheduler, 
    'callbacks':     [accuracy_cb, print_cb, chkpt_cb],
}
trainer_lstm = ptt.DeepNetTrainer(**training_parameters)


In [16]:
if False:
    trainer_lstm.fit(10, Xtrain, ytrain, valid_data=(Xvalid, yvalid), batch_size=batch_size)
else:
    trainer_lstm.load_state('/data/models/sentimento_lstm')

Resultado do treinamento na GPU:

        Start training for 10 epochs
          1:  19.8s   T: 0.69289 0.51508   V: 0.69216 0.54064 best
          2:  19.9s   T: 0.68615 0.61652   V: 0.64578 0.67128 best
          3:  19.8s   T: 0.44313 0.80380   V: 0.37075 0.84224 best
          4:  19.9s   T: 0.32085 0.86912   V: 0.32077 0.86364 best
          5:  19.9s   T: 0.27029 0.89392   V: 0.29334 0.87972 best
          6:  19.9s   T: 0.23622 0.91056   V: 0.28798 0.87756 best
          7:  19.9s   T: 0.21872 0.91748   V: 0.27990 0.88404 best
          8:  19.9s   T: 0.20478 0.92356   V: 0.28781 0.88408 
          9:  20.0s   T: 0.19705 0.92720   V: 0.28215 0.88464 
         10:  19.9s   T: 0.18264 0.93348   V: 0.29655 0.88156 
        Stop training at epoch: 10/10
        Best model was saved at epoch 7 with loss 0.27990: ../../models/sentimento_lstm
    

### Avaliação

In [17]:
if 'ModelCheckpoint' in [cb.__class__.__name__ for cb in trainer_lstm.callbacks]:
    trainer_lstm.load_state(state_fn)

rmetrics = trainer_lstm.evaluate(Xvalid, yvalid, metrics=[accuracy_cb])
print('Model validation set accuracy after training: {:.5f}'.format(rmetrics['acc']))

evaluate: 2499/2499 ok
Model validation set accuracy after training: 0.88404


## Resumo dos resultados

1. Experimento *bag of words*: 87% de acurácia
2. Experimento *word embeddings*, rede densa: 88%
3. Experimento *word embeddings*, rede convolucional: 89%
4. Experimento *word embeddings*, RNN: 87%
5. Experimento *word embeddings*, LSTM: 88.4%